In [20]:
import tdmclient.notebook
import numpy as np
await tdmclient.notebook.start()

Exception: Node lock error

In [31]:
%%run_python
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target
    SPEED=50
    PROX_FACTOR=100
    THRESHOLD_PROX=50
    if prox_horizontal[0]>THRESHOLD_PROX or prox_horizontal[1]>THRESHOLD_PROX:
        #Should turn right
        motor_left_target=(SPEED+prox_horizontal[0]//PROX_FACTOR+prox_horizontal[1]//PROX_FACTOR)
        motor_right_target=(SPEED-prox_horizontal[0]//PROX_FACTOR-prox_horizontal[1]//PROX_FACTOR)
    else:
        #####################################################Code claqué sinon fonctionne pas
        if prox_horizontal[4]>THRESHOLD_PROX:
            motor_left_target=(SPEED-prox_horizontal[4]//PROX_FACTOR-prox_horizontal[3]//PROX_FACTOR)
            motor_right_target=(SPEED+prox_horizontal[4]//PROX_FACTOR+prox_horizontal[3]//PROX_FACTOR)
        if prox_horizontal[3]>THRESHOLD_PROX:
        #Should turn left
            motor_left_target=(SPEED-prox_horizontal[4]//PROX_FACTOR-prox_horizontal[3]//PROX_FACTOR)
            motor_right_target=(SPEED+prox_horizontal[4]//PROX_FACTOR+prox_horizontal[3]//PROX_FACTOR)
    

Exception in thread Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Thread-11:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Valentin Roch\Desktop\python\lib\site-packages\tdmclient\tcp.py", line 65, in run
    self.run()
  File "C:\Users\Valentin Roch\Desktop\python\lib\site-packages\tdmclient\tcp.py", line 65, in run
        packet = self.read_packet()
  File "C:\Users\Valentin Roch\Desktop\python\lib\site-packages\tdmclient\tcp.py", line 58, in read_packet
packet = self.read_packet()
  File "C:\Users\Valentin Roch\Desktop\python\lib\site-packages\tdmclient\tcp.py", line 58, in read_packet
        raise error
  File "C:\Users\Valentin Roch\Desktop\python\lib\site-packages\tdmclient\tcp.py", line 53, in read_packet
raise error
  File "C:\Users\Valentin Roch\De

In [3]:
await tdmclient.notebook.stop()

In [4]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()

In [5]:
async def motors_command(motors):
    return {
        "motor.left.target": [motors[0]],
        "motor.right.target": [motors[1]],
    }

await node.lock()
comm= await motors_command((0,0))
print(comm)
node.send_set_variables(comm)
await node.unlock()

{'motor.left.target': [0], 'motor.right.target': [0]}


In [5]:
import numpy as np
from numpy.lib.function_base import angle

from tdmclient import ClientAsync
client = ClientAsync()


THRESHOLD_POS=1     #Has to be tuned
THRESHOLD_ANGLE=10
THRESHOLD_PROX=20
SPEED=30
P=10
D=0.001
PROX_FACTOR=1/50

#Navigation

#Output Kalman: Position, angle, vitesse, vitesse angulaire

#Output path planning: Prochaine position

def motors_command(motors):
    return {
        "motor.left.target": [motors[0]],
        "motor.right.target": [motors[1]],
    }



def angle(P1,P2):
    angle=np.arctan2(P2[1]-P1[1],P2[0]-P1[0])
    #right half
    if P2[0]>=P1[0]:
        #Up
        if P2[1]>=P1[1]:
            return angle
        #Bot
        else:
            return angle+2*np.pi
    #left half
    else:
        #Up
        if P2[1]>=P1[1]:
            return angle+np.pi
        else:
            return angle+(3/2)*np.pi


def path_side(dep, obj, pos_r):
    v_path=np.array([obj[0]-dep[0],obj[1]-dep[1]])
    v_robot=np.array([pos_r[0]-dep[0],pos_r[1]-dep[1]])
    vect_prod=np.cross(v_path,v_robot)
    #left
    if vect_prod>=0:
        return True
    #right
    else:
        return False
#return the position error
def error(pos_r, dep, obj):
    error=np.linalg.norm(np.cross(obj-dep, dep-pos_r))/np.linalg.norm(obj-dep)
    return error

#When the objectif is reached, one must receive a new objectif and the precedent objectif becomes the depart position
def reach_obj(pos_r, obj, objectif_number, obj_list):    
    if pos_r[0]>=obj[0]-THRESHOLD_POS and pos_r[0]<=obj[0]+THRESHOLD_POS:
        if pos_r[1]>=obj[1]-THRESHOLD_POS and pos_r[1]<=obj[1]+THRESHOLD_POS:
            next_obj(obj_list, objectif_number)
            return True
    else:
        return False

#When on an objectif, choose the optimal side to turn
def optimal_side(angle_r, dep, obj):
    rad=angle(dep,obj)

    #right
    if abs(angle_r-rad) <= abs(2*np.pi-angle_r+rad):
        return False
    else:
        return True

#When an objectif is reached, the robot turn on itself until it reaches the good angle.
def start_angle(angle_r, dep, obj, pos_r):
    rad = angle(dep, obj)
    #The angle is good
    if angle_r>=rad-THRESHOLD_ANGLE and angle_r<=rad+THRESHOLD_ANGLE:
        return np.array([SPEED,SPEED])
        #left
    else:
        if optimal_side(angle_r, dep, obj):
            motors=np.array([SPEED,-SPEED])
            #err_pos=0
        else:
            motors=np.array([-SPEED,SPEED])
            #err_pos=0
        
        return motors
        #err_pos

def local_avoidance():
    prox=np.array([prox_horizontal[0], prox_horizontal[4]])
    if prox_horizontal[0]>THRESHOLD_PROX:
        #Should turn right
        return np.array([int(SPEED+prox_horizontal[0]*PROX_FACTOR), int(SPEED-prox_horizontal[0]*PROX_FACTOR)])

    else:
        if prox_horizontal[4]>THRESHOLD_PROX:
        #Should turn left
            return np.array([int(SPEED-prox_horizontal[4]*PROX_FACTOR), int(SPEED+prox_horizontal[4]*PROX_FACTOR)])



#Modify dep and obj
def next_obj(obj_list, i):
    dep=obj_list[i]
    obj=obj_list[i+1]
    return dep, obj



#Je pense qu'un regulateur P devrait suffire....
def motors_corr(prev_err_pos, err_pos, dep, obj, pos_r, T):
    print('motors_corr')

    speed_corr=err_pos*P  + D*(err_pos-prev_err_pos)/T
    motors=np.array([0,0])

    if path_side(dep, obj, pos_r):
        #left
        
        motors[0]=SPEED+speed_corr
        motors[1]=SPEED-speed_corr
    else:
        motors[0]=SPEED-speed_corr
        motors[1]=SPEED+speed_corr
    

    return motors

async def prox_val(x):
    await node.lock()
    await node.wait_for_variables({str(x)})  
    await node.unlock()
    return list(node[x])


def end(objectif_number, obj_list, pos_r):
    if objectif_number==len(obj_list) and reach_obj(pos_r, obj_list[objectif_number-1], objectif_number, obj_list):
        return np.array([0,0])
    else:
        return np.array([SPEED,SPEED])
    
def navigation(pos_r, angle_r, obj_list, prev_err_pos, T, objectif_number, prox_horizontal):
    dep, obj=next_obj(obj_list, objectif_number)

    err_pos=error(pos_r, dep, obj)
    motors=motors_corr(prev_err_pos, err_pos, dep, obj, pos_r, T)
    
    rad = angle(dep, obj)
    if reach_obj(pos_r, obj, objectif_number, obj_list):
        motors=start_angle(angle_r, dep, obj, pos_r)

    
    #err_pos

    #Est-ce que le Thymio va pas vouloir foncer dans l'obstacle après l'avoir juste dépasser avec ses capteurs
    #Si oui, on le fait avancer tout droit un petit bout
    #If prox !=0
    print(prox_horizontal)
    if prox_horizontal[0]>THRESHOLD_PROX or prox_horizontal[4]>THRESHOLD_PROX:
        motors=local_avoidance()

    #return motors, err_pos, objectif_number
    #send_command_motors(motors)
    motors=end(objectif_number, obj_list, pos_r);
    return motors

In [6]:
node = await client.wait_for_node()
while 1:

    await node.lock()
    prox_horizontal= await prox_val('prox.horizontal')
    
    #Je sais pas pourquoi j'ai besoin de ces deux unlock/lock de suite    
    await node.unlock()   
    await node.lock()
    node.send_set_variables(motors_command(navigation(np.array([0,0]), 0, np.array([[0, 0], [10,0]]), 0, 0.01, 0, prox_horizontal)))
    await node.unlock()



motors_corr
[0, 0, 0, 0, 0, 3216, 3220]
motors_corr
[0, 0, 0, 0, 0, 3187, 3169]
motors_corr
[0, 0, 0, 0, 0, 3116, 3045]
motors_corr
[1701, 2073, 0, 0, 0, 2938, 2896]
motors_corr
[0, 0, 0, 0, 0, 2836, 2821]
motors_corr
[0, 0, 0, 0, 0, 2619, 2734]
motors_corr
[0, 0, 0, 0, 0, 2435, 2604]
motors_corr
[0, 0, 0, 0, 0, 2313, 2472]
motors_corr
[0, 0, 0, 0, 3512, 2388, 2362]
motors_corr
[0, 0, 0, 0, 0, 2533, 2194]
motors_corr
[0, 0, 0, 0, 0, 2332, 2139]
motors_corr
[0, 0, 0, 0, 0, 2149, 1976]
motors_corr
[0, 0, 0, 0, 0, 1992, 1823]
motors_corr
[0, 0, 0, 0, 0, 1812, 1650]
motors_corr
[0, 0, 0, 0, 0, 1617, 1444]
motors_corr
[0, 0, 0, 0, 0, 1139, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 4838, 0, 0, 4341, 0]
motors_c

motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 4903, 0, 0, 4356, 0]
motors_corr
[0, 0, 4883, 0, 0, 4067, 3995]

motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 4923, 0, 0, 1932, 2323]
motors_corr
[0, 0, 0, 0, 0, 4258, 4643]
motors_corr
[0, 0, 0, 0, 0, 3769, 4245]
motors_corr
[0, 0, 0, 0, 0, 3506, 3808]
motors_corr
[0, 0, 0, 0, 0, 3302, 3603]
motors_corr
[0, 0, 0, 0, 0, 3154, 3340]
motors_corr
[0, 0, 0, 0, 0, 2917, 3072]
motors_corr
[0, 0, 0, 0, 0, 2855, 2911]
motors_corr
[0, 0, 0, 0, 0, 2723, 2771]
motors_corr
[0, 0, 0, 0, 0, 2635, 2667]
motors_corr
[0, 0, 0, 0, 0, 2578, 2538]
motors_corr
[0, 0, 0, 0, 0, 2325, 2420]
motors_corr
[0, 0, 0, 0, 0, 2233, 2251]
motors_corr
[0, 0, 0, 0, 0, 2008, 2088]
motors_corr
[0, 0, 0, 0, 0, 1884, 1910]
motors_corr
[0, 0, 0, 0, 0, 1599, 1733]
motors_corr
[0, 0, 0, 0, 0, 1208, 1490]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr

motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 0]
motors_corr
[0, 0, 0, 0, 0, 978, 0]
motors_corr
[0, 0, 0, 0, 0, 0, 5036]
motors_corr
[0, 0, 0, 0, 0, 2997, 5044]
mot

CancelledError: 

In [35]:
await node.lock()
comm= motors_command((0,0))
print(comm)
node.send_set_variables(comm)
await node.unlock()

{'motor.left.target': [0], 'motor.right.target': [0]}


In [ ]:
import tdmclient.notebook
#import numpy as np
await tdmclient.notebook.start()

In [32]:
%%run_python
#Manual Calibration

PERIOD=10
MS_TO_S=1000

#Rotation
count=0
time=0

timer_period[0]=PERIOD
#timer_period[1]=9000


@onevent
def button_right():
    global motor_left_target, motor_right_target, count, time, timer_period
    count=1
    motor_left_target=100
    motor_right_target=-100
    timer_period[1]=9000
    
@onevent
def button_forward():
    global motor_left_target, motor_right_target, count, time, timer_period
    motor_left_target=100
    motor_right_target=100
    timer_period[1]=9000
    time=0
    count=0
#30cm en 9s
    
@onevent
def button_center():
    global count,time, PERIOD, MS_TO_S, motor_left_target, motor_right_target
    print(time*PERIOD)
    count=0
    time=0
    motor_left_target=0
    motor_right_target=0
    
@onevent
def button_backward():
    global count,time, PERIOD, MS_TO_S, motor_left_target, motor_right_target
    print(time)
    
@onevent
def timer0():
    global count, time, motor_left_target, motor_right_target
    if count==1:
        time=time+1
        if time*PERIOD>9133:
            motor_left_target=0
            motor_right_target=0

#@onevent
#def timer1():
#    global motor_left_target, motor_right_target
#    motor_left_target=0
#    motor_right_target=0
#
    
#9133 ms for one turn

DisconnectedError: TDM disconnected

In [ ]:
await tdmclient.notebook.stop()